In this experiment, you will train models to distringuish examples of two different genres of Shakespeare's plays: comedies and tragedies. (We'll ignore the histories, sonnets, etc.) Since he died four hundred years ago, Shakespeare has not written any more plays—although scraps of various other works have come to light. We are not, therefore, interested in building models simply to help categorize an unbounded stream of future documents, as we might be in other applications of text classification; rather, we are interested in what a classifier might have to tell us about what we mean by the terms “comedy” and “tragedy”.

You will start by copying and running your `createBasicFeatures` function from the experiment with movie reviews. Do the features the classifier focuses on tell you much about comedy and tragedy in general?

You will then implement another featurization function `createInterestingFeatures`, which will focus on only those features you think are informative for distinguishing between comedy and tragedy. Accuracy on leave-one-out cross-validation may go up, but it more important to look at the features given the highest weight by the classifier. Interpretability in machine learning, of course, may be harder to define than accuracy—although accuracy at some tasks such as summarization is hard enoough.

In [1]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut
import numpy as np

In [2]:
#read in the shakespeare corpus
def readShakespeare():
  raw = requests.get("https://raw.githubusercontent.com/mutherr/CS6120-PS1-data/master/shakespeare_plays.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  #remove histories from the data, as we're only working with tragedies and comedies
  corpus = [entry for entry in corpus if entry["genre"] != "history"]
  return corpus

This is where you will implement two functions to featurize the data:

In [3]:
#NB: The current contents are for testing only
#This function should return: 
#  -a sparse numpy matrix of document features
#  -a list of the correct genre for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix. 

# This function should create a feature representation using all tokens that
# contain an alphabetic character.
from sklearn.feature_extraction.text import CountVectorizer

def createBasicFeatures(corpus):
  #Your code here
    genres = []
    all_text = []
    for i in corpus:
        genres.append(i['genre'])
        all_text.append(i['text'])

    vectorizer1 = CountVectorizer()
    texts = vectorizer1.fit_transform(all_text)
    vocab = vectorizer1.get_feature_names()
    return texts, genres, vocab

# This function can add other features you want that help classification
# accuracy, such as bigrams, word prefixes and suffixes, etc.
import nltk
nltk.download("popular")

def createInterestingFeatures(corpus):
  #Your code here
    genres = []
    all_text = []   
    stop_words = [] 
    for i in corpus:
        words = []
        genres.append(i['genre'])
        words.extend([word for word in i['text'].split() if word.isalpha()])
        freq_words = nltk.FreqDist(words)
        for word in freq_words.most_common(int(len(set(words))/20)):
          stop_words.append(word[0])
        all_text.append(i['text'])

    vectorizer2 = CountVectorizer(ngram_range=(1, 2), max_features=5000,stop_words=stop_words, min_df=0.4, max_df=0.75)
    texts = vectorizer2.fit_transform(all_text)
    vocab = vectorizer2.get_feature_names()
    return texts, genres, vocab

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

In [4]:
#given a numpy matrix representation of the features for the training set, the 
# vector of true classes for each example, and the vocabulary as described 
# above, this computes the accuracy of the model using leave one out cross 
# validation and reports the most indicative features for each class
def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=LeaveOneOut())
  
  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)
  
  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  neg_class_prob_sorted = model.coef_[0, :].argsort()
  pos_class_prob_sorted = (-model.coef_[0, :]).argsort()

  termsToTake = 20
  pos_indicators = [vocab[i] for i in neg_class_prob_sorted[:termsToTake]]
  neg_indicators = [vocab[i] for i in pos_class_prob_sorted[:termsToTake]]

  return avg_score,pos_indicators,neg_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  

In [5]:
corpus = readShakespeare()

Run the following to train and evaluate two models with basic features:

In [6]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.807692
The most informative terms for pos are: ['helena', 'prospero', 'sir', 'you', 'your', 'for', 'me', 'duke', 'of', 'love', 'preserver', 'preserved', 'preserve', 'preserv', 'preservation', 'preservers', 'presents', 'presentment', 'presently', 'presenting']
The most informative terms for neg are: ['our', 'him', 'rom', 'iago', 'thy', 'ham', 'imogen', 'what', 'brutus', 'his', 'lear', 'timon', 'preservers', 'preserver', 'preserved', 'preserv', 'preservative', 'preservation', 'presents', 'presentment']
----------L2 Norm-----------
The model's average accuracy is 0.730769
The most informative terms for pos are: ['you', 'prospero', 'duke', 'helena', 'antonio', 'me', 'for', 'your', 'sir', 'ariel', 'sebastian', 'hermia', 'lysander', 'parolles', 'stephano', 'will', 'leontes', 'caliban', 'demetrius', 'love']
The most informative terms for neg are: ['ham', 'iago', 'him', 'our', 'othello', 'what', 'his', 'lear', 'imogen', 'brutus', '

Run the following to train and evaluate two models with features that are interesting for distinguishing comedy and tragedy:

In [7]:
X,y,vocab = createInterestingFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.961538
The most informative terms for pos are: ['worship', 'ship', 'fancy', 'jest', 'french', 'lover', 'captain', 'troth', 'dance', 'sheep', 'palace', 'ran', 'raise', 'rail', 'rack', 'quoth', 'abide', 'rank', 'quench', 'quarter']
The most informative terms for neg are: ['soldiers', 'slain', 'poison', 'whore', 'march', 'battle', 'ran', 'raise', 'rail', 'pomp', 'rack', 'quoth', 'quit', 'quiet', 'quench', 'quarter', 'quarrel', 'quality', 'qualities', 'push']
----------L2 Norm-----------
The model's average accuracy is 0.846154
The most informative terms for pos are: ['worship', 'ship', 'troth', 'jest', 'lover', 'fancy', 'sheep', 'fish', 'pinch', 'marriage', 'burden', 'dance', 'humour', 'colour', 'remedy', 'quoth', 'wench', 'pound', 'ladyship', 'credit']
The most informative terms for neg are: ['slain', 'poison', 'soldiers', 'whore', 'trumpet', 'march', 'serv', 'battle', 'wounds', 'warlike', 'struck', 'plague', 'lust', 'kingdom

**Basic Features**

Here, simple bag of words model is use to create a sparse matrix and this does pretty well to predict the genre of the text. With the use of min_df and max_df we can control the terms in the model to improve the accuracy, by removing terms occuring less than min_df and terms occuring more than max_df. These options are tested more thoroughly in fancy features.

**Interesting Features**

Bigram model is used. Here we have removed the stop_words which are basically the most frequent occuring terms, which in this case are the names and pronouns. And limiting the max features to 5000 gives best accuracy. Along with min_df=0.4, max_df=0.75. We see that L1 norm is capturing the features very well compared to L2 norm. L2 norm is penalizing the outliers more, hence the accuracy is lower. The feautures of L1 norm make good case for the genre class.
